In [91]:
%%capture imports
!pip install ta
!pip install plotly==5.4
!pip install pandas
!pip install pandas_datareader
!pip install yfinance
!pip install -U kaleido
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt
from ta.trend import MACD
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [82]:
tickers = ['DIS', 'NFLX', 'GOOG', 'FB', 'AMZN', 'NVDA', 'TSLA', 'MSFT', 'AAPL']
# Last 3 years closing prices starting from Jan 2, 2018.
 
now = dt.datetime.now()
start = now - dt.timedelta(days=120)

all_data = {}

def heikin_ashi(df):
    heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['HKOpen', 'HKHigh', 'HKLow', 'HKClose'])
    
    heikin_ashi_df['HKClose'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    
    for i in range(len(df)):
        if i == 0:
            heikin_ashi_df.iat[0, 0] = df['Open'].iloc[0]
        else:
            heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i-1, 0] + heikin_ashi_df.iat[i-1, 3]) / 2
        
    heikin_ashi_df['HKHigh'] = heikin_ashi_df.loc[:, ['HKOpen', 'HKClose']].join(df['High']).max(axis=1)
    
    heikin_ashi_df['HKLow'] = heikin_ashi_df.loc[:, ['HKOpen', 'HKClose']].join(df['Low']).min(axis=1)
    
    return heikin_ashi_df

for t in tickers:
   # df = pdr.get_data_yahoo(t, start, now)
    df = yf.download(t, 
                      start, 
                      now, progress=False)

    # removing all empty dates
    # build complete timeline from start date to end date
    dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
    # retrieve the dates that ARE in the original datset
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
    # define dates with missing values
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

    # add moving averages to df
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()

    # MACD
    macd = MACD(close=df['Close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)

    df['MACD'] = macd.macd()
    df['MACD_sig'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()
    ha = heikin_ashi(df)

    df = pd.merge(ha, df, how='outer', left_index=True, right_index=True)
    df = df.dropna()
    all_data[t] = df



In [90]:
all
# Show signals
last_candle_index = -2
candle_index = -1
print("Latest Day Candle: ")
print(all_data[tickers[0]].last_valid_index())
print("\nSignals:")
for t in tickers:
    df = all_data[t]
    candle = df['HKClose'].iloc[candle_index] - df['HKOpen'].iloc[candle_index]
    candle_last = df['HKClose'].iloc[last_candle_index] - df['HKOpen'].iloc[last_candle_index]
    macd = df['MACD_diff'].iloc[candle_index] 
    if candle > 0:
        if candle_last < 0 and macd < 0:
            print(t + ": No MACD Confirm")
        if candle_last < 0 and macd >= 0:
            print(t + ": BUY")
        else:
            print(t + ": HOLD")
    elif candle < 0:
        print(t + ": SELL")

Latest Day Candle: 
2021-12-17 00:00:00

Signals:
DIS: SELL
NFLX: SELL
GOOG: SELL
FB: SELL
AMZN: SELL
NVDA: SELL
TSLA: SELL
MSFT: SELL
AAPL: SELL


In [92]:
for t in tickers:
    df = all_data[t]

    # first declare an empty figure
    fig = go.Figure()

    # add subplot properties when initializing fig variable
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        vertical_spacing=0.01, 
                        row_heights=[0.4,0.1,0.1])
    # add OHLC trace on first row
    fig.add_trace(go.Candlestick(x=df.index,
                                open=df['HKOpen'],
                                high=df['HKHigh'],
                                low=df['HKLow'],
                                close=df['HKClose'], 
                                showlegend=False))

    # add moving average traces on same row
    fig.add_trace(go.Scatter(x=df.index, 
                            y=df['MA5'], 
                            opacity=0.7, 
                            line=dict(color='blue', width=2), 
                            name='MA 5'))
    fig.add_trace(go.Scatter(x=df.index, 
                            y=df['MA20'], 
                            opacity=0.7, 
                            line=dict(color='orange', width=2), 
                            name='MA 20'))

    # Plot MACD trace on 2nd row
    fig.add_trace(go.Scatter(x=df.index,
                            y=df['MACD'],
                            line=dict(color='black', width=2)
                            ), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                            y=df['MACD_sig'],
                            line=dict(color='blue', width=1)
                            ), row=2, col=1)

    colors = ['green' if val >= 0 
            else 'red' for val in df['MACD_diff']]

    fig.add_trace(go.Bar(x=df.index, 
                        y=df['MACD_diff'],
                        marker_color=colors
                        ), row=2, col=1)

    # Plot volume trace on 3rd row
    colors = ['green' if row['Open'] - row['Close'] >= 0 
            else 'red' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(x=df.index, 
                        y=df['Volume'],
                        marker_color=colors
                        ), row=3, col=1)

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MACD", showgrid=False, row=2, col=1)
    fig.update_yaxes(title_text="Volume", showgrid=False, row=3, col=1)

    # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
    fig.update_layout(height=450, width=1200, 
                    showlegend=False, 
                    title=t,
                    xaxis_rangeslider_visible=False,
                    xaxis_rangebreaks=[dict(values=dt_breaks)])

    # removing white space
    fig.update_layout(margin=go.layout.Margin(
            l=20, #left margin
            r=20, #right margin
            b=20, #bottom margin
            t=40  #top margin
        ))

    #from IPython.display import Image
    #image_bytes = fig.to_image(format='png', width=1200, height=700, scale=1)
    #display(Image(image_bytes))

    fig.show()

